# Explicabilidad en Inteligencia Artificial

## 1. Introducción

Debido al incremento exponencial del desarrollo y la integración de sistemas inteligentes en actividades de diversa naturaleza y frecuencia, la comprensión de su funcionamiento resulta fundamental por diversos motivos. 
En primer lugar si un modelo pretende **desempeñar un papel fundamental como una herramienta para ayudar a la toma de decisiones**, su modo de razonamiento debe ser fácilmente entendible por un ser humano tradicional. Así se puede analizar su **comportamiento** para observar rasgos indeseados que poder corregir, asegurar que sus pilares se encuentran alineados con los objetivos del problema que aborda, que cumple una determinada **ética** establecida en torno a las soluciones que propone, además de medidas de seguridad críticas con las que controlar la **calidad** de las mismas. 

De este modo en este notebook se pretende aplicar distintas técnicas de explicabilidad con la que comprender más detalladamente la conducta característica de los clasificadores elaborados para la detección de textos sexistas.

### 1.1. Tipos de explicabilidad

* **Modelos agnósticos locales**. Esta categoría de técnicas se pueden emplear sobre cualquier tipo de modelo de caja negra aunque únicamente se centran en una sola muestra.

## 2. Estructura del notebook

1. Introducción a la Explicabilidad en Inteligencia Artificial
2. Estructura del notebook
3. Instalación y carga de librerías
4. Lectura y carga de datos procesados
5. Técnicas de explicabilidad
6. Conclusiones

## 3. Instalación y carga de librerías

Este apartado tiene como único propósito cargar las librerías y dependencias necesarias para la ejecución de este notebook, así como las funciones propiamente desarrolladas. Previo a ello deberán ser instaladas bien ejecutando el script *setup.sh* mediante el comando `bash setup.sh` con permisos de ejecución en distribuciones Linux, o bien ejecutando el compando `pip install -r requirements.txt`.

In [1]:
%%capture
import sys
sys.path.append('../scripts')

# Import encoding functions
from encoding import *

# pandas: to read the datasets
import pandas as pd

# numpy: to work with predicted probabilities
import numpy as np

# skelearn: to encode documents
from sklearn.feature_extraction.text import CountVectorizer

# pickle: to load Logistic Regression models
import pickle 

# lime: to create local explanations
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

# random: to choose random samples
from random import randrange

## 4. Lectura y carga de datos procesados

En esta sección se pretende leer y cargar los datos de entrenamiento y test ya procesados a los que se les ha aplicado las siguientes técnicas de tratamiento de documentos:

  - Elimina URLs.
  - Elimina usuarios mencionados.
  - Elimina caracteres especiales, no alfabéticos y signos de puntuación.
  - Convierte todos los caracteres en minúsculas.

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.865 muestras**.
* Conjunto de validación: **4.296 muestras**.

In [2]:
# Read already processed EXIST datasets
train_df = pd.read_csv('../data/proc_EXIST2021_train.csv')
test_df = pd.read_csv('../data/proc_EXIST2021_test.csv')

# Show the dimensions of the datasets
print('Train dataset dimensions:', train_df.shape)
print('Test dataset dimensions:', test_df.shape)

Train dataset dimensions: (6865, 6)
Test dataset dimensions: (4296, 6)


## 5. Técnicas de explicabilidad

### 5.1. LIME

Se trata de un procedimiento que genera **explicaciones locales a una única muestra dentro de una determinada clase** cuyo objetivo consiste en determinar la relevancia de cada característica (palabra) proporcionada para su predicción. Para ello genera un **modelo lineal** que se entrena en base a perturbaciones generadas a partir de un ejemplo seleccionado y luego es evaluado dependiendo de la similitud entre las predicciones producidas y la real asociada a la entrada suministrada. Esta tarea es posible mediante el uso de la librería *lime* de Python a la que se le debe proporcionar una muestra de ejemplo junto con una función que permite obtener la predicción de las perturbaciones creadas.

#### 5.1.1. Modelo inglés de Regresión Logística

Comenzamos ejecutando este método sobre el mejor clasificador inglés de Regresión Logística encontrado cuya codificación se realizó transformando los textos en bolsas de palabras. Puesto que se trata de un método local a un ejemplo, se propone su aplicación en los conjuntos de **falsos positivos y negativos** analizados en notebooks anteriores con la idea de comprobar cuáles son los términos destacables que han liderado hacia una clasificación errónea. Como el volumen de ambos es bastante considerable tal y como se anuncia en los resultados anteriores, con 213 y 438 muestras, nos apoyaremos en los estudios efectuados en función de los intervalos de confianza para seleccionar **diez ejemplos de cada rango de forma aleatoria**.

In [3]:
# Load a Logistic Regression model specialized in English
en_lr_model = pickle.load(open('../models/multileng_lr_models/en_bag_words_model.sav', 'rb'))

# Filter the train documents by language
en_train_df = train_df[train_df['language'] == 'en']
en_train_texts = list(en_train_df['clean_text'].values)

# Filter the test documents by language
en_test_df = test_df[test_df['language'] == 'en']
en_test_texts = list(en_test_df['clean_text'].values)

# Convert train and test documents to bag of words
en_train_bag_words, en_test_bag_words = to_bag_of_words(
    train_docs=en_train_texts, 
    test_docs=en_test_texts)

# Predict over the test dataset
en_test_df['task1_pred_classes'] = en_lr_model.predict(X=en_test_bag_words)
en_test_df['task1_pred_probs'] = [max(prob) for prob in en_lr_model.predict_proba(X=en_test_bag_words)]

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

No. of false negatives: 438
No. of false positives: 213



/tmp/ipykernel_8388/2998631689.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_classes'] = en_lr_model.predict(X=en_test_bag_words)
/tmp/ipykernel_8388/2998631689.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_probs'] = [max(prob) for prob in en_lr_model.predict_proba(X=en_test_bag_words)]


Las conclusiones extraídas de la generación de explicaciones locales a un subconjunto de **falsos negativos** se presentan a continuación:

* Los falsos negativos pertenecientes al más elevado intervalo de confianza otorga un grado de **no-sexismo a palabras sin significado** como pronombres, determinantes, preposiciones y verbos auxiliares, que pueden contrarrestar el efecto de terminología sexista. 

* Aquellos cometidos bajo un rango de confianza alto demuestran un notable grado de **ironía** con terminología positiva aunque con significados profundamente sexistas.

* Finalmente los falsos negativos que se encuentran dentro del intervalo moderado de confianza están caracterizados por su alta composición de **hashtags y errores ortográficos** que dificultan su análisis e identificación automáticos.

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def predict_en_lrm_model(sample: str):

        # Train a CountVectorizer object based on train documents
        bg_vectorizer = CountVectorizer()
        bg_vectorizer.fit(raw_documents=en_train_texts)

        # Encode the provided test sample 
        encoded_sample = bg_vectorizer.transform(raw_documents=sample).toarray()
        
        # Predict the probabilities for both classes
        pred_prob = en_lr_model.predict_proba(X=encoded_sample)
        return np.array(pred_prob)

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

Replicando el mismo procedimiento anterior aunque aplicado a los **falsos positivos** se han podido descubrir las siguientes pesquisas:

* Los falsos positivos de los dos intervalos superiores de confianza **contienen noticias y reivindaciones de usuarios contra los ataques y la violencia contra la mujer**. No obstante, términos comunes como "mujeres", "feminismo", etc. son catalogados como sexistas y por ende se han categorizado bajo esta clase erróneamente.

* Los falsos positivos situados en el intervalo moderado de confianza destacan por tratar **otros tópicos** en los que aparecen también palabras de género como "mujeres", "marido", entre otras que pese a ser clasificadas como sexistas su significado no es tal. 

#### 5.1.1. Modelo español de Regresión Logística

A continuación se repite el estudio previo pero haciendo uso del mejor modelo español de Regresión Logística con sus correspondientes documentos en dicho idioma. En los siguientes resultados se puede apreciar que existen 462 documentos sexistas no detectados y 199 textos no sexistas erróneamente clasificados.

In [6]:
# Load a Logistic Regression model specialized in Spanish
es_lr_model = pickle.load(open('../models/multileng_lr_models/es_bag_words_model.sav', 'rb'))

# Filter the train documents by language
es_train_df = train_df[train_df['language'] == 'es']
es_train_texts = list(es_train_df['clean_text'].values)

# Filter the test documents by language
es_test_df = test_df[test_df['language'] == 'es']
es_test_texts = list(es_test_df['clean_text'].values)

# Convert train and test documents to bag of words
es_train_bag_words, es_test_bag_words = to_bag_of_words(
    train_docs=es_train_texts, 
    test_docs=es_test_texts)

# Predict over the test dataset
es_test_df['task1_pred_classes'] = es_lr_model.predict(X=es_test_bag_words)
es_test_df['task1_pred_probs'] = [max(prob) for prob in es_lr_model.predict_proba(X=es_test_bag_words)]

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

No. of false negatives: 462
No. of false positives: 199



/tmp/ipykernel_8388/1758952928.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_classes'] = es_lr_model.predict(X=es_test_bag_words)
/tmp/ipykernel_8388/1758952928.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_probs'] = [max(prob) for prob in es_lr_model.predict_proba(X=es_test_bag_words)]


Las deducciones descubiertas tras la visualización de diez muestras positivas clasificadas erróneamente seleccionadas aleatoriamente por cada intervalo de confianza, como se ha efectuado con el clasificador inglés, han sido idénticamente similares a las observadas previamente.

Por otro lado en relación a los falsos positivos, es decir ejemplos negativos identificados como positivos, además de compartir las características descubiertas anteriormente también cabe destacar la **sexualización por parte del modelo de prendas femeninas**, como la falda, además de la no comprensión de terminología latinoamericana puesto que no se les asigna ninguna clase o valor por lo que se consideran neutros.